In [109]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import glob

In [110]:
imgs=[]
for image_path in glob.glob("./Pokemon/*"):
    image=cv2.imread(image_path,cv2.IMREAD_COLOR)
    image = cv2.normalize(image.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
    imgs.append(image)

In [111]:
poke_images=np.array(imgs)

In [112]:
batch_size=len(poke_images)
height=40
width=40
channels=3

In [113]:
#I'm following the Lenet5 Values for some of the filters and kernels
input_size=792
batch_size=24
training_epochs=1000
display_step=100
X=tf.placeholder(tf.float32,shape=(None,height,width,channels))
filter1=tf.Variable(tf.truncated_normal([5,5,channels,6],stddev=0.035))#6 3x3xchannels filters for RGB
bias1=tf.Variable(tf.truncated_normal([6],stddev=0.035))
ex=tf.constant(0.1,shape=[5,height,width,channels])
expected=tf.nn.conv2d(ex,filter1,strides=[1,1,1,1],padding="SAME")
print(expected.get_shape())
conv1=tf.nn.conv2d(X,filter1,strides=[1,1,1,1],padding="SAME")
conv1=tf.nn.bias_add(conv1,bias1)
conv1=tf.nn.relu(conv1)
pool1=tf.nn.max_pool(conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
poolexpected=tf.nn.max_pool(expected,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
print(poolexpected.get_shape())
filter2=tf.Variable(tf.truncated_normal([5,5,6,16],stddev=0.035))
bias2=tf.Variable(tf.truncated_normal([16],stddev=0.035))
expected2=tf.nn.conv2d(poolexpected,filter2,strides=[1,1,1,1],padding="SAME")
print(expected2.get_shape())
conv2=tf.nn.conv2d(pool1,filter2,strides=[1,1,1,1],padding="SAME")

conv2=tf.nn.bias_add(conv2,bias2)
conv2=tf.nn.relu(conv2)
pool2=tf.nn.max_pool(conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
poolexpected2=tf.nn.max_pool(expected2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
print(poolexpected2.get_shape())
defilter1=tf.Variable(tf.truncated_normal([5,5,6,16],stddev=0.035))
deconv1=tf.nn.conv2d_transpose(pool2,defilter1,tf.stack([tf.shape(X)[0],10,10,6]),strides=[1,1,1,1],padding="SAME")
debias1=tf.Variable(tf.truncated_normal([6],stddev=0.035))
deconv1=tf.nn.bias_add(deconv1,debias1)
deconv1=tf.nn.sigmoid(deconv1)
depool1=tf.image.resize_nearest_neighbor(deconv1,size=tf.stack([20,20]))
debias2=tf.Variable(tf.truncated_normal([3],stddev=0.035))
defilter2=tf.Variable(tf.truncated_normal([5,5,channels,6],stddev=0.035))
deconv2=tf.nn.conv2d_transpose(depool1,defilter2,tf.stack([tf.shape(X)[0],20,20,3]),strides=[1,1,1,1],padding="SAME")
deconv2=tf.nn.bias_add(deconv2,debias2)
deconv2=tf.nn.sigmoid(deconv2)
depool2=tf.image.resize_nearest_neighbor(deconv2,size=tf.stack([40,40]))


(5, 40, 40, 6)
(5, 20, 20, 6)
(5, 20, 20, 16)
(5, 10, 10, 16)


In [114]:
cost=tf.reduce_mean(tf.pow(depool2-X,2))
optimizer=tf.train.AdamOptimizer(learning_rate=0.05).minimize(cost)

In [115]:
init=tf.global_variables_initializer()
sess=tf.InteractiveSession()
sess.run(init)

In [ ]:
#This training part is entirely copied

total_batch = int(input_size/batch_size)
# Training cycle
for epoch in range(training_epochs):
    # Loop over all batches
    start = 0; end = batch_size
    for i in range(total_batch-1):
        index = np.arange(start, end)
        np.random.shuffle(index)
        batch_xs = poke_images[index]
        start = end; end = start+batch_size
        # Run optimization op (backprop) and loss op (to get loss value)
        _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs})
    # Display logs per epoch step
    if ((epoch == 0) or (epoch+1) % display_step == 0) or ((epoch+1) == training_epochs):
        print ('Epoch: {0:05d}   loss: {1:f}'.format(epoch+1, c))

print("Optimization Finished!")


Epoch: 00001   loss: 0.110709
